Importing libraries

In [10]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\storage_key.json"

Reading csv file containing product_name and url

In [155]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\data.csv")
data

,product_name,url
0,NIKE Womens W Revolution 7 Running Shoe,https://m.media-amazon.com/images/I/71cy1dbMh5...
1,Puma Men's T-Shirt,https://m.media-amazon.com/images/I/51V8KtLHdm...
2,Asus Vivobook 15,https://m.media-amazon.com/images/I/71Ue-ZFSPJ...
3,Brown Zip Wallet,https://m.media-amazon.com/images/I/910SYv5lcO...
4,Apple Iphone 15 pro max,https://m.media-amazon.com/images/I/81fxjeu8fd...


Storing url column in data_url

In [156]:
data_url = data['url']
data_url

0    https://m.media-amazon.com/images/I/71cy1dbMh5...
1    https://m.media-amazon.com/images/I/51V8KtLHdm...
2    https://m.media-amazon.com/images/I/71Ue-ZFSPJ...
3    https://m.media-amazon.com/images/I/910SYv5lcO...
4    https://m.media-amazon.com/images/I/81fxjeu8fd...
Name: url, dtype: object

Function to check if url contains a image or not

In [139]:
import requests
def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   r = requests.head(image_url)
   if r.headers["content-type"] in image_formats:
      print(r.headers["content-type"])
      return True
   return False

In [175]:
import webcolors
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

requested_colour = (119, 172, 152)
actual_name, closest_name = get_colour_name(requested_colour)

In [198]:

def detect_properties_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    # print("Properties:",props.dominant_colors.colors)
    # scr = [col for col in props.dominant_colors.colors]
    # print(scr)
    for color in props.dominant_colors.colors:
        try:
            r = int(color.color.red)
            g = int(color.color.green)
            b = int(color.color.blue)
            print(f"color: {get_colour_name((r,g,b))}")
        except Exception as e:
            print("gone from here",e)
            continue
        
detect_properties_uri(data_url[3])

color: (None, 'white')
color: (None, 'rosybrown')
color: (None, 'silver')
color: (None, 'darkslategray')
color: (None, 'antiquewhite')
color: (None, 'tan')
color: (None, 'rosybrown')
color: (None, 'dimgray')
color: (None, 'silver')
color: (None, 'rosybrown')


Empty String variable to store results

In [141]:
txt = ""

Function to extract information from image

In [142]:
def extract_info(uri):
    global txt
    try:
        if(is_url_image(uri)==True):
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri
            
            #For retriving Brand
            res_logo = client.logo_detection(img)
            logos = res_logo.logo_annotations
            logo = [log.description for log in logos]
            txt = txt + f"Brand: None, " if len(logo)==0 else txt + f"Brand: {logo[0]}, "
            
            #For retriving Labels
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]
            txt = txt + f"Description: None, " if len(label)==0 else txt + f"Description: {str(label)} ,"
            
            #For retriving Text
            res_text = client.document_text_detection(img)
            texts = res_text.text_annotations
            text = [t.description for t in texts]
            txt = txt + f"Text: None" if len(text)==0 else txt + f"Text: {text[0]}"
        
        else:
            txt = "Please enter a valid URL that contains Image"
    except Exception as e:
        txt = f"Please Enter a Valid URL of Image"
        return

Creating new dataframe to save results

In [143]:
result = data
result

,product_name,url
0,black color,https://upload.wiktreimedia.org/wikipedia/comm...


Iterating through function to extract features of image and adding it to result dataframe

In [144]:
for i in range(len(data_url)):
    txt = ""
    extract_info(data_url[i])
    result.loc[i, "Extracted_Details_from_Image"] = txt
    

Resultant Dataframe

In [145]:
result

,product_name,url,Extracted_Details_from_Image
0,black color,https://upload.wiktreimedia.org/wikipedia/comm...,Please Enter a Valid URL of Image


Saving Dataframe to CSV file

In [146]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)

Uploading csv file to Cloud Stroage

In [2]:
storage_cli = storage.Client()
bucket = storage_cli.get_bucket("bucket-shreyash")
blob = bucket.blob("Extracted_features_from_image.csv")
blob.upload_from_filename("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
print("completed")

completed
